### Entrenamiento, Predicción y Evaluación de los modelos de ensamble

#### **Entrenamiento**

Modelo Bagging (1):

Modelos Boosting (3):

---

#### **Predicción**

A un paso (Bagging)

A múltiples pasos (Bagging)

A un paso (Boosting x3)

A múltiples pasos (Boosting x3)

---

#### **Analisis de resultados**

Métricas a evaluar: 
1. sMAPE
2. WAPE

---